# **DL Assignment 2 Set 4 - Question2**
## **DL Group 215**
###Nareskumar P (2020FC04122),
###Kommajyosula VNS Kanth (2020FC04120),
###Tejesh Dola (2020FC04459)

**Import Libraries**

In [1]:
import numpy as np
import pandas as pd
import torch
import torchvision
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

**Defining Functions**

In [2]:
def generate_sequence(n): #Generating a bit sequence of length n parity(sequence)
    bit_seq = []
    for i in range(n):
        bit = np.random.randint(0,2)
        bit_seq.append(bit)
    return bit_seq

def parity(sequence): #Checks the parity of a bit sequence (Returns 1 if even, 0 if odd)
    k = 1
    for bit in sequence:
        if(bit == 0):
            bit = -1
        k*= bit
    if(k == -1):
        return 0
    return 1

def generate_set(m,n): #Creates a dataset of m points which are bit sequences of length n
    #m = no of bit sequences
    #n = length of bit sequences
    dataset = []
    for i in range(m):
        dataset.append(generate_sequence(n))
    return dataset

def cross_validate(model, X_test, y_test): #Checks the accuracy of "deep_net" neural network on a test set of size m
    count = 0
    correct = 0
    for i in range(len(X_test)):
        output = model(X_test[i])
        if(y_test[i]==torch.round(output)):
            correct +=1
        count+=1
    return correct/count

def training_accuracy(X,y,model): #Checks the accuracy of "deep_net" neural network on training set (X,y)
    correct = 0
    for i in range(len(X)):
        output = model(X[i])
        if(y[i]==torch.round(output)):
            correct += 1
    return correct/len(X)

generate_sequence(n) - Generating a bit sequence of length n

parity(sequence) - Checks the parity of a bit sequence (Returns 1 if even, 0 if odd)

generate_set(m,n) - Creates a dataset of m points which are bit sequences of length n

cross_validate(m,n,net) - Checks the accuracy of "deep_net" neural network on a test set of size m

training_accuracy(X,y,net) - Checks the accuracy of "deep_net" neural network on training set (X,y)

**Sanity Check**

In [3]:
dataset = generate_set(5,6)

print("Dataset: \n")

for seq in dataset:
    print("Bit Sequence: ",seq,"\nParity:",parity(seq),"\n")

Dataset: 

Bit Sequence:  [1, 1, 0, 1, 1, 0] 
Parity: 1 

Bit Sequence:  [1, 0, 0, 1, 1, 0] 
Parity: 0 

Bit Sequence:  [1, 1, 0, 1, 0, 0] 
Parity: 0 

Bit Sequence:  [1, 1, 0, 0, 1, 0] 
Parity: 0 

Bit Sequence:  [0, 1, 1, 1, 1, 1] 
Parity: 0 



**Neural Network design**

Creating a 6 layered network model for parity function using a neural network.

In [4]:
class Deep_Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(64,32)
        self.fc2 = nn.Linear(32,16)
        self.fc3 = nn.Linear(16,8)
        self.fc4 = nn.Linear(8,4)
        self.fc5 = nn.Linear(4,2)        
        self.fc6 = nn.Linear(2,1)
        self.drop = nn.Dropout(0.3)
        
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.drop(self.fc3(x)))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = F.sigmoid(self.fc6(x))
        return x
        
deep_net = Deep_Net()
deep_net.to(device)
print(deep_net)

Deep_Net(
  (fc1): Linear(in_features=64, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=8, bias=True)
  (fc4): Linear(in_features=8, out_features=4, bias=True)
  (fc5): Linear(in_features=4, out_features=2, bias=True)
  (fc6): Linear(in_features=2, out_features=1, bias=True)
  (drop): Dropout(p=0.3, inplace=False)
)


**Training and Cross Validation**

In [5]:
def train_and_cross_validate(model, X_train, y_train, epochs=150, alpha=1e-3):
    
    # m - size of the training set
    optimizer = optim.Adam(model.parameters(), lr = alpha)
    for epoch in range(epochs):
        for i in range(len(X_train)):
            X = X_train[i]
            y = y_train[i]

            model.zero_grad()

            y = y.view(-1,1)
            output = model(X.view(-1,64))

            criterion = nn.BCELoss()

            loss = criterion(output,y)

            loss.backward()

            optimizer.step()
    
        if((epoch+1) in [1,5,10,25,50,75,100,150]):
            print("Epoch:", epoch+1)
            print("Training Loss:",loss.item())
            print("Training Accuracy:", training_accuracy(X_train,y_train,model))
            print("CrossVal Accuracy:", cross_validate(model, X_test,y_test),"\n")

**Training Set of Size 3000**

In [6]:
X_train = generate_set(3000,64)
y_train = [parity(x) for x in X_train]

X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train)
    
X_train, y_train = X_train.to(device), y_train.to(device)
    
X_test = generate_set(100,64)
y_test = [parity(x) for x in X_test]
    
X_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(y_test)
    
X_test, y_test = X_test.to(device), y_test.to(device)

In [7]:
train_and_cross_validate(deep_net,X_train,y_train)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 1
Training Loss: 0.7330824136734009
Training Accuracy: 0.5
CrossVal Accuracy: 0.48 

Epoch: 5
Training Loss: 0.6941338777542114
Training Accuracy: 0.5
CrossVal Accuracy: 0.48 

Epoch: 10
Training Loss: 0.6940410733222961
Training Accuracy: 0.5
CrossVal Accuracy: 0.48 

Epoch: 25
Training Loss: 0.6940410733222961
Training Accuracy: 0.5
CrossVal Accuracy: 0.48 

Epoch: 50
Training Loss: 0.6940410733222961
Training Accuracy: 0.5
CrossVal Accuracy: 0.48 

Epoch: 75
Training Loss: 0.6940410733222961
Training Accuracy: 0.5
CrossVal Accuracy: 0.48 

Epoch: 100
Training Loss: 0.6940410733222961
Training Accuracy: 0.5
CrossVal Accuracy: 0.48 

Epoch: 150
Training Loss: 0.6940410733222961
Training Accuracy: 0.5
CrossVal Accuracy: 0.48 



**Training Set of Size 6000**

In [8]:
X_train = generate_set(6000,64)
y_train = [parity(x) for x in X_train]

X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train)
    
X_train, y_train = X_train.to(device), y_train.to(device)
    
X_test = generate_set(100,64)
y_test = [parity(x) for x in X_test]
    
X_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(y_test)
    
X_test, y_test = X_test.to(device), y_test.to(device)


In [9]:
train_and_cross_validate(deep_net,X_train,y_train)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1806: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch: 1
Training Loss: 0.682487428188324
Training Accuracy: 0.49816666666666665
CrossVal Accuracy: 0.51 

Epoch: 5
Training Loss: 0.6819507479667664
Training Accuracy: 0.49816666666666665
CrossVal Accuracy: 0.51 

Epoch: 10
Training Loss: 0.6819507479667664
Training Accuracy: 0.49816666666666665
CrossVal Accuracy: 0.51 

Epoch: 25
Training Loss: 0.6819507479667664
Training Accuracy: 0.49816666666666665
CrossVal Accuracy: 0.51 

Epoch: 50
Training Loss: 0.6819507479667664
Training Accuracy: 0.49816666666666665
CrossVal Accuracy: 0.51 

Epoch: 75
Training Loss: 0.6819507479667664
Training Accuracy: 0.49816666666666665
CrossVal Accuracy: 0.51 

Epoch: 100
Training Loss: 0.6819507479667664
Training Accuracy: 0.49816666666666665
CrossVal Accuracy: 0.51 

Epoch: 150
Training Loss: 0.6819507479667664
Training Accuracy: 0.49816666666666665
CrossVal Accuracy: 0.51 



We observed that training set of both 3000 or 6000 and not improving the cross validation accuracy. This may be because the weights converge to a local minimum.

The parity function isn't properly captured by the neural network (due to 2^64 possibilities) and it's clear from the cross validation scores that the predictions are only as good as random guessing when it comes to an unseen test set (Close to 50% accuracy)